In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Image dimensions
IMG_HEIGHT = 256
IMG_WIDTH = 256

# Paths
images_dir = 'D:\BTP\Dataset\set_btp\set_1\set_1'
masks_dir = 'D:\BTP\Dataset\masks\mask_1'

# Load and preprocess dataset
def load_data():
    X, Y = [], []
    for fname in os.listdir(images_dir):
        image_path = os.path.join(images_dir, fname)
        mask_path = os.path.join(masks_dir, fname.replace('.jpg', '_mask.png'))

        if not os.path.exists(mask_path):
            continue

        # Load and resize images and masks
        img = cv2.imread(image_path)
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)

        X.append(img / 255.)
        Y.append(mask)

    X = np.array(X)
    Y = np.array(Y)
    Y = np.expand_dims(Y, axis=-1)

    return X, Y

X, Y = load_data()

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build a simple U-Net model
def build_unet(input_shape):
    inputs = Input(input_shape)

    c1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(16, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D()(c1)

    c2 = Conv2D(32, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(32, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D()(c2)

    c3 = Conv2D(64, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(64, 3, activation='relu', padding='same')(c3)

    u1 = UpSampling2D()(c3)
    merge1 = concatenate([u1, c2])
    c4 = Conv2D(32, 3, activation='relu', padding='same')(merge1)
    c4 = Conv2D(32, 3, activation='relu', padding='same')(c4)

    u2 = UpSampling2D()(c4)
    merge2 = concatenate([u2, c1])
    c5 = Conv2D(16, 3, activation='relu', padding='same')(merge2)
    c5 = Conv2D(16, 3, activation='relu', padding='same')(c5)

    outputs = Conv2D(4, 1, activation='softmax')(c5)

    model = Model(inputs, outputs)
    return model

# Compile and train the model
model = build_unet((IMG_HEIGHT, IMG_WIDTH, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_val, y_val))

# Save the model
model.save('unet_model.h5')


In [2]:
import numpy, cv2, PIL, tensorflow, sklearn

print("NumPy version:", numpy.__version__)
print("OpenCV version:", cv2.__version__)
print("Pillow version:", PIL.__version__)
print("TensorFlow version:", tensorflow.__version__)
print("Scikit-Learn version:", sklearn.__version__)


NumPy version: 1.26.0
OpenCV version: 4.11.0
Pillow version: 10.4.0
TensorFlow version: 2.18.0
Scikit-Learn version: 1.5.1
